# Interactive development workflow
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas environment, enabling quick iterations to analyze and tune the model before going to Big Data.

To run the example of the parametric simulation you need to run one of the following sections to build the model executeables.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../../src')

In [ ]:
import OMPython

In [ ]:
import tempfile
import json
import importlib
from OMPython import ModelicaSystem
from ModelicaModels import BouncingBall
import DyMat

## Either Build model and run simulation with interactive OMC session...
This section demonstrates how to simulate and of the model using the [OMPython API](https://www.openmodelica.org/doc/OpenModelicaUsersGuide/latest/ompython.html) ***with*** a connection to omc enabling the the manipulation of parameters through `setParameters`.

In [ ]:
# mod = BouncingBall.instantiatemodel()  # Does NOT connect to omc
modelname = BouncingBall.modelName
# xmlFilePath = os.path.join(SETUP_DIR, 'ModelicaModels', 'build', modelname)
mod = ModelicaSystem(  # connects to omc
        fileName=os.path.dirname(BouncingBall.__file__) + '/' + modelname+".mo",
        modelName=modelname,
    )

In [ ]:
sim_options_d = mod.getSimulationOptions()
sim_options_d['stopTime'] = 2
mod.setSimulationOptions(sim_options_d)

In [ ]:
mod.getSimulationOptions()

In [ ]:
%%time
resfilename = os.path.join(tempfile.gettempdir(), 'b.mat')
mod.setParameters("e=0.2")  # works in interactive mode only
mod.simulate(resultfile=resfilename, simflags=None)

In [ ]:
# Needs connection to omc
vars = mod.getSolutions()
vars

In [ ]:
%%time
# TODO Loading results (from mat-file) is very slow
data = mod.getSolutions(varList=list(vars), resultfile=resfilename)
ts_df = pd.DataFrame(data).T
ts_df.columns = vars
ts_df

In [ ]:
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))
ax.plot(ts_df['time'], ts_df['h'])
plt.tight_layout()

## Or build a model without connecting to OMC
This section demonstrate how to build a model using modelica `*.mos` script files ***without*** connection to omc.

In [ ]:
BouncingBall.create_mos_file()  # creates a *.mos file next to the location of BouncingBall

In [ ]:
r = BouncingBall.run_mos_file()  # builds the model executable in the current working directory

In [ ]:
r

## Parametric simulation without connecting to omc
Parameters can be provided to precompiled models as arguments - without connecting or even requiring omc. `instantiatemodel('BouncingBall')` looks for the model (*BouncingBall_init.xml*) in the current working directory (when created by the section above) or in *src/ModelicaModels/build/BouncingBall* when created via `python setup.py build`.

See also: https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/scripting_api.html#simulation-parameter-sweep

In [ ]:
from ModelicaModels import BouncingBall

In [ ]:
parameters_var_df = pd.DataFrame(columns=['run_key', 'modifiers'], data=[
    ['r1', {'e': 0.7, 'stopTime': 3.0}],
    ['r2', {'e': 0.5, 'stopTime': 3.0}],
    ['r3', {'e': 0.9, 'stopTime': 3.0}],
])
parameters_var_df['modifiers'] = parameters_var_df['modifiers'].apply(json.dumps)
parameters_var_df

### Sequential execution

In [ ]:
mod = BouncingBall.instantiatemodel('BouncingBall')

In [ ]:
mod.overridevariables

In [ ]:
mod.getParameters()

In [ ]:
def dymat2pandas(dm, block, names) -> pd.DataFrame:
    ts_df = pd.DataFrame(dm.getVarArray(names)).T
    #ts_df['time'] = dm.abscissa(2)
    ts_df.columns=['time'] + names
    return ts_df


def run_sim_parametric(pdf, modelwrapper_name=None, model_name=None, res_vars=None, use_local=True) -> pd.DataFrame:
    """Simulation of a single run. The unique run identifier is in the column run_key."""
    if use_local:
        modelwrapper = importlib.import_module(modelwrapper_name)
        mod = modelwrapper.instantiatemodel(model_name)
    else:
        mod = instantiatemodel(model_name, use_local=False)
    log_str = ''
    print(pdf)
    if 'time' in pdf.columns:
        pdf = pdf.sort_values('time').reset_index()
    temp_dir = tempfile.gettempdir()
    grp = pdf['run_key'].iloc[0]
    resfilename = mod.modelName + '_' + grp + '.mat'
    resfilepathname = os.path.join(temp_dir, resfilename)
    # mod.setParameters(pdf['modifiers'].iloc[0])  # Seems to be not implemented in OMPython
    # We might need to encode the dictonary as json when using pyspark
    if 'modifiers' in pdf.columns:
        overridevariables = json.loads(pdf['modifiers'].iloc[0])
        if len(overridevariables) > 0:
            mod.overridevariables = overridevariables
        pdf.drop('modifiers', axis=1, inplace=True)
    inputs_required = set(mod.getInputs().keys())
    if len(inputs_required) > 0:
        inputs_available = set(pdf.columns)
        if inputs_available.intersection(inputs_required) != inputs_required:
            raise ValueError(
                'The required input series not provided in the DataFrame.\nRequired: {0}. Provided:{1}'.format(
                    str(inputs_required), str(inputs_available)
                )
            )
        # print(str(['time']+list(mod.getInputs().keys())))
        pdf = pdf[['time']+list(mod.getInputs().keys())]
        # Write csv file and tell the model class
        mod.csvFile = os.path.join(temp_dir, mod.modelName + '_' + grp + '_inputs.csv')
        pdf.to_csv(mod.csvFile, sep=',', line_terminator=',\n')
        mod.inputFlag = True
        if 'stopTime' in mod.overridevariables:
            stopTime = float(mod.overridevariables.pop('stopTime'))
        else:
            stopTime = float(pdf['time'].max())
        # Run the simulation
        log_str += str(mod.simulate(
            resultfile=resfilepathname,
            simflags=None,
            #overrideaux='variableFilter="'+'|'.join(list(res_vars))+'"'
            overrideaux='stopTime={0},'.format(stopTime) + 'variableFilter="'+'|'.join(res_vars)+'"'
        ))
    else:
        # Run the simulation
        log_str += str(mod.simulate(
            resultfile=resfilepathname,
            simflags=None
        ))
    print(log_str)
    if len(mod.csvFile) > 0:
        os.remove(mod.csvFile)
    # Collect results
    if isinstance(res_vars, tuple):
        res_vars = list(res_vars)
    try:
        dm = DyMat.DyMatFile(resfilepathname)
        ts_df = dymat2pandas(dm, 2, res_vars)
        #os.remove(resfilepathname)
    except Exception as e:
        ts_df = pd.DataFrame(columns=['time'] + res_vars, data=[[-1.0 ,0.0, 0.0]])
    ts_df.columns = ['time'] + res_vars
    ts_df['run_key'] = grp
    # print(ts_df.head(3))
    return ts_df


def get_sim_dist_func(modelwrapper, modelname, run_fun=run_sim_parametric, res_vars=None, use_local=True):
    """Return the pandas (udf) function to simulate a set of runs."""
    modelwrapperName = modelwrapper.__name__
    def run_sim_dist(pdf) -> pd.DataFrame():
        return run_fun(
            pdf,
            modelwrapper_name=modelwrapperName, model_name=modelname, res_vars=res_vars,
            use_local=use_local
        )
    return run_sim_dist

In [ ]:
%%time
ts_all_df = parameters_var_df.groupby(['run_key']).apply(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'])
    )

In [ ]:
ts_all_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
p_df = ts_all_df[[
    'time', 'run_key', 'h'
]].reset_index(drop=True).drop_duplicates(subset=['time', 'run_key'])
p_df.pivot(index='time',columns='run_key', values='h').plot()

### Parallel execution with Spark

In [ ]:
parameters_var_sdf = spark.createDataFrame(parameters_var_df)
parameters_var_sdf.show()

In [ ]:
parameters_var_sdf.toPandas()['modifiers'].iloc[0]

### Parallel execution

In [ ]:
from ModelicaRuntimeTools import addpymodules

In [ ]:
sc = spark.sparkContext

In [ ]:
addpymodules([os.path.dirname(DyMat.__file__)], 'mdymat.zip', sc=sc, dironly=True)
addpymodules([os.path.dirname(OMPython.__file__)], 'mOMPython.zip', sc=sc, dironly=True)

In [ ]:
# Run the model wrapper module in the current namespace
modelwrapper_pyfile = BouncingBall.__file__

In [ ]:
%run -i $modelwrapper_pyfile

In [ ]:
instantiatemodel

In [ ]:
from pyspark.sql import types as T, functions as F
res_schema = T.StructType([
    T.StructField("time", T.DoubleType(), True),
    T.StructField("h", T.DoubleType(), True),
    T.StructField("v", T.DoubleType(), True),
    T.StructField("run_key", T.StringType(), True),
])
# Running the parametric simulation
ts_sim_sdf = parameters_var_sdf.groupby(['run_key']).applyInPandas(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'], use_local=False), schema=res_schema,
    ).cache()

In [ ]:
ts_sim_sdf.show()

In [ ]:
for n, run_key in enumerate(parameters_var_df['run_key'].unique()):
    ts_run_sim_pdf = ts_sim_sdf.where(F.col('run_key') == run_key).toPandas()
    plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['h'])
plt.show()